<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [1]:
# Your Code Starts Here

In [2]:
from sklearn.model_selection import train_test_split


In [3]:
def load_quickdraw10(path):
  X = data['arr_0']
  y = data['arr_1']
  X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.20, random_state=42)
  return X_train, y_train, X_test, y_test

In [5]:
import numpy as np
data = np.load('quickdraw10.npz')

BadZipFile: ignored

In [ ]:
X_train, y_train, X_test, y_test = load_quickdraw10(data)

#quickly visualize the shape: 
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
from sklearn import preprocessing
X_train = preprocessing.normalize(X_train)
X_test = preprocessing.normalize(X_test)

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import os
import datetime

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

In [ ]:
def train_test_model(hparams):
  model = Sequential([
                      Dense(hparams[HP_NUM_UNITS], activation='relu'),
                      Dense(10, activation='softmax') 
  ])
  opt_name = hparams[HP_OPTIMIZER]
  lr= hparams[HP_LEARNING_RATE]

  if opt_name == 'sgd':
    opt = tf.keras.optimizers.SGD(learning_rate=lr)
  elif opt_name == 'adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
  else:
    raise Exception('Unrecognized optimizer. Must be either "sgd" or "adam"')
  model.compile(optimizer=opt, 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
  )
  model.fit(X_train, y_train, epochs=5)
  _, val_acc = model.evaluate(X_test, y_test)
  return val_acc

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0 

for num_units in HP_NUM_UNITS.domain.values:
  for learning_rate in tf.linspace(HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value, num=3):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_LEARNING_RATE:  float(learning_rate),
          HP_OPTIMIZER: optimizer
      }

      run_name = f"run-{session_num}"
      print(f"--- Starting trial: {run_name}")
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

In [ ]:
%tensorboard --logdir logs/hparam_tuning

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 